In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gran

In [3]:
pixnet_corpus_path = "E:/LangOn/Corpus/top_articles_200k.jl.gz"

In [14]:
import gzip
import json
from collections import Counter
from bs4 import BeautifulSoup
    

In [19]:
counter = Counter()
pbar = gran.tqdm()
with gzip.open(pixnet_corpus_path, "rt", encoding="UTF-8") as fin:    
    ln = fin.readline()
    while True:
        if not ln: break        
        dat = json.loads(ln)
        site_category = dat.get("site_category", "")
        counter.update([site_category])
        ln = fin.readline()
        pbar.update()
pbar.close()

In [69]:
sum(counter.values())

208743

In [22]:
counter.most_common(10)

[('美味食記', 84169),
 ('國外旅遊', 49996),
 ('國內旅遊', 34559),
 ('美容彩妝', 15971),
 ('食譜分享', 13558),
 ('時尚流行', 10308),
 ('生活綜合', 46),
 ('攝影寫真', 45),
 ('親子育兒', 36),
 ('活動紀錄', 9)]

In [31]:
foods = []
pbar = gran.tqdm()
with gzip.open(pixnet_corpus_path, "rt", encoding="UTF-8") as fin:    
    ln = fin.readline()
    while True:
        if not ln: break
        dat = json.loads(ln)                
        if dat.get("site_category") == '美味食記':                        
            ptag = BeautifulSoup(dat.get("body", ""))
            foods.append(ptag.text)        
            pbar.update()
        ln = fin.readline()        
pbar.close()

In [33]:
gran.install_data_cache("pixnet_cache")
foods_path = gran.get_cache_path("pixnet_cache", "foods.pkl")

In [35]:
import pickle
with open(foods_path, "wb") as fout:
    pickle.dump(foods, fout)

In [41]:
# character count
"Character count: {:,}".format(sum([len(x) for x in foods]))

'Character count: 154,690,439'

In [37]:
foods[0]

'\n最近好像又到了尋覓伴手禮的時間了\n之前都會買月餅回老家，但發現月餅份量也多，\n熱量也是比較高，經常會銷路比較沒那麼好\n\n這次改吃帶點養生材料的餅乾-千味軒 千味餅\n分享包200g NT$120元\n這是來自台南的手工餅乾\n\n研發的老闆是20歲出頭的年輕人，是個迷途青年回家鄉台南奮鬥，用不斷嘗試、研發的精神，\n結合了台、日、法式的創新技術製法創造出特殊的口感\n說服經營傳統麵包店的母親而創立的自己的品牌\n起緣滿讓人感動的耶XD這種奮鬥不懈的精神讓人敬佩!\n<下圖來自官方圖片>\n\n看到食材內竟然有棉花糖耶!!!\n用棉花糖來做餅乾真讓人期待XD\n<下圖來自官方圖片>\n\n另外有沒有看到材料內有非常多乾果\n包含：枸杞、葡萄乾、青提子乾、蔓越莓乾、橘子皮等\n感覺都是非常養生的食材阿!\n\n分享包 220g\n保存期限：1個月(未拆封)\n\n拆封後要趕快吃完，放在陰涼處\n外包裝有夾鏈袋的設計，方便沒吃完密封好貼心!\n\n我的是綜合口味喔!\n裡面包含 原味、黑糖\n每個餅都有單獨的衣服好好的收著\n\n拆開包裝就微微散發香氣\n看切面是個很特別的餅乾耶!!\n而且可以看到他們使用的乾果類食材扎扎實實包覆在裡面\n\n黑色的是黑糖口味，我覺得黑糖口味喜愛黑糖的人會喜歡阿!\n他的口感有點鬆鬆軟軟帶點Q彈的感覺\n\n我個人偏愛原味口味阿!!多董愛黑糖口味!!\n剛好沒互搶XD\n冰起來之後吃，還會有不同風味喔!\n長溫下吃軟軟QQ的~冰起來吃後口感更佳耶\n\n這個餅乾不會太甜喔!!感覺甜度有設計調整過\n適合現代人低糖低油的健康取向\n這包給長輩吃也很棒!\n裡面包含的乾果類也是有益好物阿!\n\n其實阿血之前對養生類餅乾有點刻板映像\n會怕葡萄乾跟枸杞味道有些害怕排斥\n但這款竟然我一開始吃沒感覺到在吃這些我害怕的乾果耶XD\n\n中秋節快到啦!\n這個適合送給長輩、爸媽\n\n我的是分享包裝喔!!\n他們也有推出送禮的禮盒包裝\n小禮盒350g\n禮盒組650g\n\n中秋禮盒推薦!!\n禮盒中秋優惠 買十送一\n\n宅配運費180元，滿2200元免運費\n\n金沢·千味軒\n地址：台南市西港區新復里溪埔寮32之6號\n訂購專線：0966732197\n營業時間：週一～週五 10:00～18:00\n粉絲頁\n\n更多宅配美食

# Find Patterns

In [45]:
import pandas as pd

In [46]:
annot_path = gran.get_cache_path('deep_annot', 'annotations.csv')
annot = pd.read_csv(annot_path)

In [51]:
pats = annot.lexical_unit[annot.form == 3.2].tolist()

In [63]:
import re
pats_counter = {}

In [64]:
for pat_x in gran.tqdm(pats):
    pat_re = re.compile(pat_x)
    for article_x in foods:    
        pats_counter[pat_x] = pats_counter.get(pat_x, 0) + len(pat_re.findall(article_x))    

In [67]:
Counter(pats_counter).most_common(30)

[('(好)+吃(唷|呦)?', 193244),
 ('～+', 185502),
 ('朋友', 178590),
 ('XD+', 77263),
 ('(真的)?(超|超級|很|非常)好吃', 40415),
 ('一?整個就?', 38776),
 ('P\\.?S\\.?|p\\.?s\\.?', 36284),
 ('還不(錯|賴)說?', 29179),
 ('Q彈', 28326),
 ('單點', 24150),
 ('(每到|每次來)?必(吃|點)+', 23003),
 ('(真的)?(超|超級|很|非常)愛', 19182),
 ('入口即化', 17937),
 ('沒有?(想|料)到', 17637),
 ('(真的)?(好|很|超|超級|非常)棒', 17000),
 ('很是?不錯', 14248),
 ('立(馬|刻|即)', 12560),
 ('也(很|蠻|満)?不錯', 12267),
 ('嚼勁', 11168),
 ('(調|拿捏)?(得|的)?剛剛好', 11091),
 ('fu', 10640),
 ('><', 9927),
 ('大力?推(薦|荐)?', 9114),
 ('重點(其實|應該)?是', 8497),
 ('雷', 8274),
 ('(超|就很|很)?夠味', 8167),
 ('(很|狠)?大推', 7929),
 ('低消', 7883),
 ('一整個', 7178),
 ('ㄟ', 7013)]